In [7]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import numpy as np
import jax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from timecast.utils.numpy import ecdf
from timecast.utils.losses import MeanSquareError
import torch
import matplotlib

plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
results = {}
for i in tqdm.tqdm([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]):
    results[i] = {}
    for j in np.linspace(-8, 0.75, 36):
        if j == -8 or j == 0:
            name = "{}_{}".format(i, int(j))
        else:
            name = ".".join("{0}_{1:.2f}".format(i, j).split("0."))
        with open("../tmp/batch_{}.pkl".format(name), "rb") as f:
            results[i][j] = pickle.load(f)

In [9]:
data = {}
for batch_key, batch in results.items():
    data[batch_key] = {}
    for lr_key, lr in batch.items():
        key = list(lr.keys())[-1]
        data[batch_key][lr_key] = lr[key]["avg_mse"]

In [10]:
data = pd.DataFrame.from_dict(data)

In [11]:
data

,1,2,4,8,16,32,64,128,256,512,1024
-8.00,3.2136214,3.219556,3.2231734,3.2251909,3.2262523,3.2267907,3.2270544,3.2271767,3.2272332,3.2272618,3.227276
-7.75,3.2068446,3.2150168,3.2204506,3.223698,3.2254846,3.2264152,3.2268772,3.227093,3.2271929,3.2272434,3.2272687
-7.50,3.198238,3.20879,3.216361,3.2212949,3.2241938,3.225767,3.226566,3.226945,3.2271216,3.227211,3.2272556
-7.25,3.187493,3.2008803,3.2107003,3.217651,3.2221017,3.2246728,3.22603,3.2266853,3.2269952,3.2271538,3.2272317
-7.00,3.1734462,3.1911883,3.2035117,3.2125661,3.2189062,3.222891,3.2251213,3.2262359,3.226774,3.2270522,3.2271907
-6.75,3.1537652,3.1789355,3.1948848,3.2061162,3.214419,3.2201564,3.2236373,3.2254722,3.226389,3.2268739,3.227118
-6.50,3.1251557,3.1623247,3.1844242,3.1985567,3.2087495,3.2163067,3.2213502,3.2242174,3.2257297,3.2265625,3.2269888
-6.25,3.0845752,3.1386664,3.170889,3.1898658,3.2023077,3.2114966,3.218133,3.222268,3.2246363,3.2260273,3.226763
-6.00,3.0312164,3.1054509,3.1522589,3.1793618,3.1954465,3.2062964,3.2141838,3.2195017,3.2229095,3.2251294,3.2263727
-5.75,2.9671562,3.062486,3.126661,3.1657312,3.18808,3.2014627,3.2102222,3.216104,3.2203946,3.2236905,3.225712
